# Sparse Hebbian Learning with full homeostasis : testing different rescaling functions

In this notebook, we test the convergence of SparseNet as a function of different parameters tuning the quantization. These parameters only influence the way we select dictionary elements and thus the homeostasis.



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=2, suppress=True)

In [3]:
from shl_scripts.shl_experiments import SHL_set
tag = 'rescaling'

list_figures = []
list_figures = ['show_dico', 'show_Pcum']
list_figures = ['show_Pcum']
display_variables = ['error', 'logL', 'qerror', 'aerror', 'MI', 'cputime']
display_variables = ['error', 'logL', 'perror', 'MC', 'cputime']

In [4]:
!ls -l {experiments.shl.cache_dir}/{tag}*
!ls -l {experiments.shl.cache_dir}/{tag}*lock*

ls: {experiments.shl.data_cache}/{tag}*: No such file or directory
ls: {experiments.shl.data_cache}/{tag}*lock*: No such file or directory


## different rescaling values

In [ ]:
opts = dict(homeo_method='HEH', verbose=1)
experiments = SHL_set(opts, tag=tag)

experiments.scan(variable='C', list_figures=list_figures, display='')
plt.show()
for display_variable in display_variables:
    fig, ax = experiments.scan(variable='C', list_figures=[], display='final', display_variable=display_variable)
    #plt.show()

Extracting data..loading the data called : cache_dir/data_data
Data is of shape : (65520, 324) - done in 0.17s.
No cache found cache_dir/rescaling - C=0.75000_dico.pkl: Learning the dictionary with algo = mp 
 Training on 65520 patches
[dict_learning] Iteration   1 /   1025 (elapsed time:   1s,   0mn   1s)
Iteration  33 /   1025 (elapsed time:  56s,   0mn  56s)
Iteration  65 /   1025 (elapsed time:  111s,   1mn  51s)
Iteration  97 /   1025 (elapsed time:  165s,   2mn  45s)
Iteration  129 /   1025 (elapsed time:  220s,   3mn  40s)
Iteration  161 /   1025 (elapsed time:  275s,   4mn  35s)
Iteration  193 /   1025 (elapsed time:  331s,   5mn  31s)
Iteration  225 /   1025 (elapsed time:  387s,   6mn  27s)
Iteration  257 /   1025 (elapsed time:  442s,   7mn  22s)
Iteration  289 /   1025 (elapsed time:  498s,   8mn  18s)
Iteration  321 /   1025 (elapsed time:  553s,   9mn  13s)
Iteration  353 /   1025 (elapsed time:  607s,  10mn   7s)
Iteration  385 /   1025 (elapsed time:  661s,  11mn   1s)


## different quantization parameters

with a fixed $C$, we now change the number of quantization steps used to compute the histogram:

In [ ]:
opts = dict(homeo_method='HEH', verbose=1)
experiments = SHL_set(opts, tag=tag)

experiments.scan(variable='nb_quant', list_figures=list_figures, display='')
plt.show()
for display_variable in display_variables:
    fig, ax = experiments.scan(variable='nb_quant', list_figures=[], display='final', display_variable=display_variable)
    #plt.show()

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts